SECCION VI - SITUACIÓN DE LA RELACIÓN DE PAREJA/INGRESOS Y RECURSOS

In [ ]:
import pandas as pd
import re
pd.options.display.max_columns = None

In [ ]:
df = pd.read_csv('./conjunto_de_datos_endireh_2021_csv/conjunto_de_datos_TB_SEC_IV/conjunto_de_datos/conjunto_de_datos_TB_SEC_IV.csv', encoding='latin-1')
df

In [ ]:
# df.describe()

In [ ]:
# df.info()

In [ ]:
df = df.astype(str)

In [ ]:
# df.info()

In [ ]:
df.replace(r'\r$','',inplace=True, regex=True)

In [ ]:
df.head()

Creación de diccionario

In [ ]:
catal = pd.read_csv('./conjunto_de_datos_endireh_2021_csv/conjunto_de_datos_TB_SEC_IV/diccionario_de_datos/diccionario_de_datos_TB_SEC_IV.csv', encoding='latin-1', usecols = ['NOMBRE_CAMPO','NEMONICO'])
catal

In [ ]:
preg =dict(zip(catal.NEMONICO, catal.NOMBRE_CAMPO))

Renombrar df

In [ ]:
df = df.rename(columns = preg)

In [ ]:
df.head()

In [ ]:
#df['Situación laboral':'Aporte económico por parte de la pareja o expareja de la informante']

In [ ]:
factor = df.filter(like = 'Factor', axis=1)
factor.head(2)

In [ ]:
#pareja = df.filter(like='Aporte económico por parte de la pareja o expareja de la informante', axis=1)
#pareja.head()

In [ ]:
idmujer = df.filter(like='Identificador', axis=1)
idmujer.head(2)

In [ ]:
trabaja = df.filter(like='Situación laboral', axis=1)
trabaja

In [ ]:
df48 = df.loc[:,'Aporte económico por parte de la pareja o expareja de la informante':'Ingreso por otro medio ']
#4.8. Actualmente, ¿usted recibe dinero...

In [ ]:
df48.head()

In [ ]:
# df48.info()

In [ ]:
autonomia = df.filter(like='Autonomía para usar los recursos monetarios en lo que se desee', axis=1)
autonomia[:1]

In [ ]:
# dfg=df48.melt()

In [ ]:
# dfg.variable.unique()

In [ ]:
salario = df.filter(like='Salario', axis=1)
salario.head(5)

In [ ]:
temp = df.filter(like='Temporalidad', axis=1)
temp[:1]

Mujeres que trabajan y cuánto ganan


In [ ]:
trabing = pd.concat([idmujer,factor, trabaja, salario, autonomia, df48], axis=1)
trabing.head()

# Invertir nombres de columnas para usar catalogos

In [ ]:
pregInv = dict(map(reversed, preg.items()))

In [ ]:
trabing = trabing.rename(columns=pregInv)

In [ ]:
trabing[:1]

In [ ]:
import platform
import glob 

catalogos = {}
mainF = "./conjunto_de_datos_endireh_2021_csv/conjunto_de_datos_TB_SEC_IV"

for cat in glob.glob(os.path.join(mainF, "catalogos/*.csv")):
  if platform.system() == 'Darwin':
    name = re.findall(r"catalogos/(.*)\.", cat)[0]
  else:
    name = re.findall(r"catalogos\\(.*)\.", cat)[0]
  tempDF = pd.read_csv(cat, engine='python', encoding='latin-1')
  tempDF.replace(r"\r","", inplace=True, regex=True)
  tempDF = tempDF.astype(str)
  tempDF = tempDF.apply(lambda x: x.str.strip())
  tempDict = dict(zip(tempDF[name], tempDF['descrip']))
  catalogos[name] = tempDict
  # print(tempDF)

In [ ]:
for col in trabing.columns:
  if col != 'P4_2' and col != 'P4_7_AB':
    trabing[col].replace(catalogos[col], inplace=True)


In [ ]:
for col in ['P4_2', 'P4_7_AB']:
  trabing[col] = pd.to_numeric(trabing[col], errors='coerce')

In [ ]:
trabing

In [ ]:
trabing.info()

In [ ]:
trabing = trabing.rename(columns=preg)

In [ ]:
trabing[:1]

In [ ]:
trabing.shape

In [ ]:
edad = pd.read_csv('./conjunto_de_datos_endireh_2021_csv/conjunto_de_datos_TSDem/conjunto_de_datos/conjunto_de_datos_TSDem.csv', encoding='latin-1')

In [ ]:
edad = edad.astype(str)

In [ ]:
edad.replace(r'\r$','',inplace=True, regex=True)
edad.head()

In [ ]:
edadd = edad[['EDAD', 'ID_PER']]

In [ ]:
edadd.head()

In [ ]:
edadd = edadd.rename(columns={"ID_PER": "Identificador de la mujer", "EDAD": "Edad"}, errors="raise")
edadd.head()

In [ ]:
result = pd.merge(edadd, trabing, on="Identificador de la mujer")
result.head()

In [ ]:
result.Edad = pd.to_numeric(result.Edad, errors='coerce')

In [ ]:
result.info()

# result ya limpio

In [ ]:
result

In [ ]:
import altair as alt
alt.data_transformers.disable_max_rows()

In [ ]:
alt.Chart(result).mark_bar().encode(
    alt.X("Edad:Q", bin=True),
    y='count()',
)

In [ ]:
result.columns[:5].tolist()+['Salario', 'Cantidad de la aportación económica']

In [ ]:
resultMelt2 = result.melt(
  id_vars=result.columns[:10].tolist()
)
resultMelt2

In [ ]:
resultMelt2.to_csv('./resultMelt2.csv', encoding='latin-1')

Mujeres que no trabajan y tienen autonomía, medir su ingresos por otros medios

In [ ]:
autonSI = resultMelt2.loc[resultMelt2['Autonomía para usar los recursos monetarios en lo que se desee'] == 'Sí']
autonSI

In [ ]:
labSI = autonSI.loc[autonSI['Situación laboral'] == 'Sí']
labSI

In [ ]:
alt.Chart(labSI).mark_bar().encode(
    x=alt.X('count(value)'),
    y=alt.Y('variable:N', sort='-x'),
    color='variable'
)

In [ ]:
resultMelt = result.melt(
  id_vars=result.columns[:5].tolist()+['Salario', 'Cantidad de la aportación económica']
)
resultMelt

In [ ]:
filtroSI = resultMelt.loc[resultMelt.value == 'Sí']

In [ ]:
#Calcular con altair

alt.Chart(filtroSI).mark_bar().encode(
    x=alt.X('count(value)', stack="normalize"),
    y=alt.Y('variable:N', sort='-x'),
    color='variable'
)

In [ ]:
# Calcular con pandas y luego graficar

graph = filtroSI.groupby('variable').count().reset_index()
graph

In [ ]:
alt.Chart(graph).mark_bar().encode(
    x=alt.X('value:Q'),
    y=alt.Y('variable:N', sort='-x'),
    color='variable'
)